<a href="https://colab.research.google.com/github/geoskimoto/SNOTEL_ErrorAndOutlier_Tools/blob/main/LogicalTestTools_03_09_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To get this ready for analysis:

*   Create additional logical tests: 
    * Scatter plot of SWE vs. Precip (should be line with slope of 1, or at least never changing)
    * Plot for precip (never   should be <0).  Can probabjust use dmp for this check.
    * Plot the incremental SWE, SNWD, and PREC and look for outliers/very large changes
    *SWE decreasing when temp > 0 deg C


*   Add single line double mass plot

Longer term:

Convert all these functions into a class(es) and clean up organization and code





# Import and Data Munging

### Library imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
import statsmodels.api as sm
from functools import reduce

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



### Data Import and Prep

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/My Drive/Projects/Work/SiteComparisons/Data

/content/gdrive/My Drive/Projects/Work/SiteComparisons/Data


In [ ]:
ls

Data_parsing.ipynb     OR_WY1990-2020.xlsx    WA_WY1990-2020.xlsx
OR_1991-2021_raw.xlsx  WA_1991-2001_raw.xlsx
OR_1991-2021.xlsx      WA_1991-2021.xlsx


In [ ]:
WA_Sites = pd.read_excel('WA_WY1990-2020.xlsx')
# pd.to_datetime(WA_Sites['Date'])
WA_Sites.set_index('Date',inplace=True)
# WA_Sites.sample(5)

OR_Sites = pd.read_excel('OR_WY1990-2020.xlsx')
# pd.to_datetime(OR_Sites['Date'])
OR_Sites.set_index('Date',inplace=True)
# OR_Sites.sample(5)

ORWA_Sites = pd.merge(OR_Sites,WA_Sites,left_index=True,right_index=True)
ORWA_Sites.head(1)

,Aneroid Lake #2 (302) Precipitation Accumulation (in) Start of Day Values,Aneroid Lake #2 (302) Precipitation Increment (in),Aneroid Lake #2 (302) Precipitation Increment - Snow-adj (in),Aneroid Lake #2 (302) Snow Depth (in) Start of Day Values,Aneroid Lake #2 (302) Snow Water Equivalent (in) Start of Day Values,Annie Springs (1000) Precipitation Accumulation (in) Start of Day Values,Annie Springs (1000) Precipitation Increment (in),Annie Springs (1000) Precipitation Increment - Snow-adj (in),Annie Springs (1000) Snow Depth (in) Start of Day Values,Annie Springs (1000) Snow Water Equivalent (in) Start of Day Values,Anthony Lake (1245) Precipitation Accumulation (in) Start of Day Values,Anthony Lake (1245) Precipitation Increment (in),Anthony Lake (1245) Precipitation Increment - Snow-adj (in),Anthony Lake (1245) Snow Depth (in) Start of Day Values,Anthony Lake (1245) Snow Water Equivalent (in) Start of Day Values,Arbuckle Mtn (304) Precipitation Accumulation (in) Start of Day Values,Arbuckle Mtn (304) Precipitation Increment (in),Arbuckle Mtn (304) Precipitation Increment - Snow-adj (in),Arbuckle Mtn (304) Snow Depth (in) Start of Day Values,Arbuckle Mtn (304) Snow Water Equivalent (in) Start of Day Values,Bald Mtn AM OR (447) Precipitation Accumulation (in) Start of Day Values,Bald Mtn AM OR (447) Precipitation Increment (in),Bald Mtn AM OR (447) Precipitation Increment - Snow-adj (in),Bald Mtn AM OR (447) Snow Depth (in) Start of Day Values,Bald Mtn AM OR (447) Snow Water Equivalent (in) Start of Day Values,Barney Creek (1246) Precipitation Accumulation (in) Start of Day Values,Barney Creek (1246) Precipitation Increment (in),Barney Creek (1246) Precipitation Increment - Snow-adj (in),Barney Creek (1246) Snow Depth (in) Start of Day Values,Barney Creek (1246) Snow Water Equivalent (in) Start of Day Values,Bear Flat Meadow AM (1291) Precipitation Accumulation (in) Start of Day Values,Bear Flat Meadow AM (1291) Precipitation Increment (in),Bear Flat Meadow AM (1291) Precipitation Increment - Snow-adj (in),Bear Flat Meadow AM (1291) Snow Depth (in) Start of Day Values,Bear Flat Meadow AM (1291) Snow Water Equivalent (in) Start of Day Values,Bear Grass (1166) Precipitation Accumulation (in) Start of Day Values,Bear Grass (1166) Precipitation Increment (in),Bear Grass (1166) Precipitation Increment - Snow-adj (in),Bear Grass (1166) Snow Depth (in) Start of Day Values,Bear Grass (1166) Snow Water Equivalent (in) Start of Day Values,...,Tinkham Creek (899) Precipitation Accumulation (in) Start of Day Values,Tinkham Creek (899) Precipitation Increment (in),Tinkham Creek (899) Precipitation Increment - Snow-adj (in),Tinkham Creek (899) Snow Depth (in) Start of Day Values,Tinkham Creek (899) Snow Water Equivalent (in) Start of Day Values,Touchet (824) Precipitation Accumulation (in) Start of Day Values,Touchet (824) Precipitation Increment (in),Touchet (824) Precipitation Increment - Snow-adj (in),Touchet (824) Snow Depth (in) Start of Day Values,Touchet (824) Snow Water Equivalent (in) Start of Day Values,Trinity (1171) Precipitation Accumulation (in) Start of Day Values,Trinity (1171) Precipitation Increment (in),Trinity (1171) Precipitation Increment - Snow-adj (in),Trinity (1171) Snow Depth (in) Start of Day Values,Trinity (1171) Snow Water Equivalent (in) Start of Day Values,Trough (832) Precipitation Accumulation (in) Start of Day Values,Trough (832) Precipitation Increment (in),Trough (832) Precipitation Increment - Snow-adj (in),Trough (832) Snow Depth (in) Start of Day Values,Trough (832) Snow Water Equivalent (in) Start of Day Values,Upper Wheeler (841) Precipitation Accumulation (in) Start of Day Values,Upper Wheeler (841) Precipitation Increment (in),Upper Wheeler (841) Precipitation Increment - Snow-adj (in),Upper Wheeler (841) Snow Depth (in) Start of Day Values,Upper Wheeler (841) Snow Water Equivalent (in) Start of Day Values,Waterhole (974) Precipitation Accumulation (in) Start of Day Values,Waterhole (974) Precipita

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

ORWA_Precip_Acc = ORWA_Sites[ORWA_Sites.columns[::5]]
stations = [ORWA_Precip_Acc.columns[i].rsplit(' ', 7)[0] for i in range(0, len(ORWA_Precip_Acc.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Precipitation Accumulation (in)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_Precip_Acc.columns = header

ORWA_Precip_Incr = ORWA_Sites[ORWA_Sites.columns[1::5]]
stations = [ORWA_Precip_Incr.columns[i].rsplit(' ', 3)[0] for i in range(0, len(ORWA_Precip_Incr.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Precipitation Increment (in)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_Precip_Incr.columns = header
# ORWA_Precip_Incr.columns = stations

ORWA_Precip_Incr_sa = ORWA_Sites[ORWA_Sites.columns[2::5]]
stations = [ORWA_Precip_Incr_sa.columns[i].rsplit(' ', 5)[0] for i in range(0, len(ORWA_Precip_Incr_sa.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Precipitation Increment - Snow-adj (in)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_Precip_Incr_sa.columns = header

ORWA_SNWD = ORWA_Sites[ORWA_Sites.columns[3::5]]
stations = [ORWA_SNWD.columns[i].rsplit(' ', 7)[0] for i in range(0, len(ORWA_SNWD.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Snow Depth (in)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_SNWD.columns = header

ORWA_SWE = ORWA_Sites[ORWA_Sites.columns[4::5]]
stations = [ORWA_SWE.columns[i].rsplit(' ', 8)[0] for i in range(0, len(ORWA_SWE.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Water Equivalent (in)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_SWE.columns = header
# ORWA_SWE.columns = stations


ORWA_Sites2 = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [ORWA_Precip_Acc,ORWA_Precip_Incr,ORWA_Precip_Incr_sa,ORWA_SNWD,ORWA_SWE])
ORWA_Sites2.head(1)


station,Aneroid Lake #2 (302),Annie Springs (1000),Anthony Lake (1245),Arbuckle Mtn (304),Bald Mtn AM OR (447),Barney Creek (1246),Bear Flat Meadow AM (1291),Bear Grass (1166),Beaver Reservoir (331),Big Red Mountain (341),Big Sheep AM (1229),Bigelow Camp (343),Billie Creek Divide (344),Blazed Alder (351),Blue Mountain Spring (357),Bourne (361),Bowman Springs (362),Buckskin Lake AM (1292),Call Meadows AM (1289),Cascade Summit (388),Chemult Alternate (395),Clackamas Lake (398),Clear Lake (401),Cold Springs Camp (406),County Line (422),Cox Flat AM (1273),Crazyman Flat (1010),Daly Lake (434),Derr. (440),Diamond Lake (442),Eilertson Meadows (464),Emigrant Springs (470),Finley Corrals AM (1274),Fish Creek (477),Fish Lk. (479),Fourmile Lake (483),Gerber Reservoir (945),Gold Center (494),Government Corrals (1250),Greenpoint (504),...,Mowich (941),Mt. Tebo (1126),Muckamuck (1259),Olallie Meadows (672),Paradise (679),Park Creek Ridge (681),Pepper Creek (1104),Pigtail Peak (692),Pinto Rock (1263),Pope Ridge (699),Potato Hill (702),Quartz Peak (707),Rainy Pass (711),Rex River (911),Salmon Meadows (728),Sasse Ridge (734),Satus Pass (1231),Sawmill Ridge (1068),Sentinel Butte (1043),Sheep Canyon (748),Skate Creek (1257),Skookum Creek (912),Sourdough Gulch (985),Spencer Meadow (776),Spirit Lake (777),Spruce Springs (984),Stampede Pass (788),Stevens Pass (791),Surprise Lakes (804),Swamp Creek (975),Swift Creek (1012),Thunder Basin (817),Tinkham Creek (899),Touchet (824),Trinity (1171),Trough (832),Upper Wheeler (841),Waterhole (974),Wells Creek (909),White Pass E.S. (863)
parameter,Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),...,Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in)
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1990-10-01,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN

# Functions

### Timeseries plotting tool

In [ ]:
def plotData(stations, parameter_of_interest):

  fig = go.Figure()
 
  for i in stations:

    fig.add_trace(go.Scatter(
      x=ORWA_Sites2.index,
      y=ORWA_Sites2[i][parameter_of_interest],
      mode='lines',
      name=i,
      hovertext = ORWA_Sites2[i][parameter_of_interest],   
  ))


  fig.update_xaxes(title_text= 'Date')
  fig.update_yaxes(title_text= parameter_of_interest)

  fig.update_layout(
   height=800,
   width=1100,
  )

  fig.show()


In [ ]:
# stations = ['Corral Pass (418)', 'Morse Lake (642)', 'Pigtail Peak (692)']

# plotData(stations, parameter_of_interest= 'Water Equivalent (in)')

### View missing data for specified station

In [ ]:
def missingData(station, parameter_of_interest):
  fig, ax = plt.subplots(figsize=(10,150))
  sns.heatmap(ORWA_Sites2[f'{station}'][[parameter_of_interest]].isnull(), cbar=False)

In [ ]:
# missingData('Morse Lake (642)', 'Precipitation Accumulation (in)')

In [ ]:
# filter = ORWA_Sites2['Morse Lake (642)'][['Precipitation Accumulation (in)']].isnull()
# ORWA_Sites2.loc[:,('Morse Lake (642)', 'Precipitation Accumulation (in)')]

# ORWA_Sites2.isin()

In [ ]:
# filter = ORWA_Sites2.loc[:,('Morse Lake (642)', 'Precipitation Accumulation (in)')] <= 0

# pd.DataFrame(ORWA_Sites2['Morse Lake (642)'][filter].index)

### Find erroneous decreases in precip accum:

In [ ]:
def decreasePrecip(station):
  filter = ORWA_Sites2.loc[:,(f'{station}', 'Precipitation Increment (in)')] < 0

  return pd.DataFrame(ORWA_Sites2['Morse Lake (642)'][filter])


In [ ]:
decreasePrecip('Morse Lake (642)')

NameError: ignored

### SWE vs. Precip

In [ ]:
def SWEvsPrecip(station):  #If you set month to a default, python sees it as a key-value argument which will break *args.

#Create dataframe with WTEQ and Precip Accum for selected station
  df = ORWA_Sites2.loc[:, (f'{station}')].loc[:, ('Water Equivalent (in)','Precipitation Accumulation (in)')]  
  df.reset_index(inplace=True)
  pd.to_datetime(df.loc[:,'Date'])

#Mask to select only data between Nov - Apr
  winter_filter = ~df.loc[:,'Date'].dt.month.between(5,10)
  df2 = df.where(winter_filter).dropna()
  # df2.set_index('Date', inplace = True)

#Add column specifying the water year (Oct 1 - Sep 30) of each record
  df2.reset_index(inplace=True)  #Have to reset the index so the dates become just a regular datetime object to get .dt.year to work.
  pd.to_datetime(df2.loc[:,'Date'])
  df2.loc[:,'water_year'] = df2.loc[:,'Date'].dt.year.where(df2.loc[:,'Date'].dt.month < 10, df2.loc[:,'Date'].dt.year + 1)
  df2.loc[:,'water_year'] = list(map(lambda x: str(x), df2.loc[:,'water_year']))

#Line plot
  fig = px.line(df2, x="Precipitation Accumulation (in)",hover_name=df2.loc[:,'Date'], y="Water Equivalent (in)", color='water_year')

  fig.update_layout(
   height=800,
   width=1100)

  fig.show()




In [ ]:
SWEvsPrecip('Morse Lake (642)')